In [1]:
using DataFrames

using Tables

In [2]:
n = 10^2

ro_x_co_x_an = DataFrame(
    "In"=>rand(1:9, n),
    "Fl"=>rand(1.0:9, n),
    "Ch"=>rand('a':'z', n),
    "St4"=>[join(rand('a':'z', 4)) for _ in 1:n],
    "St8"=>[join(rand('a':'z', 8)) for _ in 1:n],
)

;

What is the best way to read one DataFrame column?

In [3]:
# Everything in `()` is evaluated before the trial, making this a bad benchmark.
@btime $(ro_x_co_x_an[!, "St4"])

# Evaluate the global mutable variable (interpolate) so that the compiler knows more.
@btime $ro_x_co_x_an[!, "St4"]

# `view` allocates and is slower (a surprise).
@btime view($ro_x_co_x_an, !, "St4")

;

  2.083 ns (0 allocations: 0 bytes)
  79.588 ns (0 allocations: 0 bytes)
  152.402 ns (1 allocation: 48 bytes)


What is the best way to read two DataFrame columns?

In [4]:
# `zip` DataFrame.
function zi(da)
    
    for (a, b) in zip(da[!, "St4"], da[!, "St8"])
        
    end
    
end

# `eachrow` DataFrame.
function ea(da)
    
    for (a, b) in eachrow(da[!, ["St4", "St8"]])
        
    end
    
end

# `zip` method allocates more but is faster.

@btime zi($ro_x_co_x_an)

@btime ea($ro_x_co_x_an)

;

  10.041 μs (504 allocations: 17.31 KiB)
  14.250 μs (220 allocations: 7.98 KiB)


In [10]:
# Make `Matrix` and `zip`.
function mzi(da)
    
    ma = Matrix(da)
    
    for (a, b) in zip(ma[:, 4], ma[:, 5])
        
    end
    
end

# Make `Matrix` and `eachrow`.
function mea(da)
    
    ma = Matrix(da)
    
    for (a, b) in eachrow(ma[:, [4, 5]])
        
    end
    
end

# `Matrix`ing is slower...

@btime mzi($ro_x_co_x_an)

@btime mea($ro_x_co_x_an)

;

@code_warntype mea(ro_x_co_x_an)

  19.833 μs (813 allocations: 31.20 KiB)
  22.042 μs (910 allocations: 31.20 KiB)
MethodInstance for mea(::DataFrame)
  from mea(da) in Main at In[10]:13
Arguments
  #self#::Core.Const(mea)
  da::DataFrame
Locals
  @_3::Union{Nothing, Tuple{Any, Int64}}
  ma::Matrix
  @_5::Any
  b::Any
  a::Any
Body::Nothing
1 ─       (ma = Main.Matrix(da))
│   %2  = ma::Matrix
│   %3  = Base.vect(4, 5)::Vector{Int64}
│   %4  = Base.getindex(%2, Main.:(:), %3)::Matrix
│   %5  = Main.eachrow(%4)::Base.Generator{Base.OneTo{Int64}}
│         (@_3 = Base.iterate(%5))
│   %7  = (@_3 === nothing)::Bool
│   %8  = Base.not_int(%7)::Bool
└──       goto #4 if not %8
2 ┄ %10 = @_3::Tuple{Any, Int64}
│   %11 = Core.getfield(%10, 1)::Any
│   %12 = Base.indexed_iterate(%11, 1)::Any
│         (a = Core.getfield(%12, 1))
│         (@_5 = Core.getfield(%12, 2))
│   %15 = Base.indexed_iterate(%11, 2, @_5)::Any
│         (b = Core.getfield(%15, 1))
│   %17 = Core.getfield(%10, 2)::Int64
│         (@_3 = Base.iterate(%5, 

Does breaking the functions into smaller ones improve the performance?

In [6]:
# The part of the code to make a `Matrix`.
function mt(da)
    
    Matrix(da)
    
end

@btime mt($ro_x_co_x_an)

;

  2.593 μs (105 allocations: 5.86 KiB)


In [11]:
# The part of the code to `zip`.
function mazi(ma)
    
    for (a, b) in zip(ma[:, 4], ma[:, 5])
        
    end
    
end

# The part of the code to `eachrow`.
function maea(ma)
    
    for (a, b) in eachrow(ma[:, [4, 5]])
        
    end
    
end

ma = Matrix(ro_x_co_x_an)

@btime mazi($ma)

@btime maea($ma)

;

@code_warntype maea(ma)

  303.336 ns (2 allocations: 1.75 KiB)
  332.392 ns (2 allocations: 1.84 KiB)
MethodInstance for maea(::Matrix{Any})
  from maea(ma) in Main at In[11]:11
Arguments
  #self#::Core.Const(maea)
  ma::Matrix{Any}
Locals
  @_3::Union{Nothing, Tuple{SubArray{Any, 1, Matrix{Any}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}}, true}, Int64}}
  @_4::Tuple{Base.OneTo{Int64}, Int64}
  b::Any
  a::Any
Body::Nothing
1 ─ %1  = Base.vect(4, 5)::Vector{Int64}
│   %2  = Base.getindex(ma, Main.:(:), %1)::Matrix{Any}
│   %3  = Main.eachrow(%2)::Base.Generator{Base.OneTo{Int64}, Base.var"#240#241"{Matrix{Any}}}
│         (@_3 = Base.iterate(%3))
│   %5  = (@_3 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_3::Tuple{SubArray{Any, 1, Matrix{Any}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}}, true}, Int64}
│   %9  = Core.getfield(%8, 1)::SubArray{Any, 1, Matrix{Any}, Tuple{Int64, Base.Slice{Base.OneTo{Int64}}}, true}
│   %10 = Base.indexed_iterate(%9, 1)::Tuple{Any, 

Sum of the parts (3 us + 0.3 us = 3.3 us) were less than their combined (20 us).

Does substituting these parts improve the performance?

In [ ]:
# Use a function to barrier the code that makes a `Matirx`.

function mt_mzi(da)
    
    ma = mt(da)
    
    for (a, b) in zip(ma[:, 4], ma[:, 5])
        
    end
    
end

function mt_mea(da)
    
    ma = mt(da)
    
    for (a, b) in eachrow(ma[:, [4, 5]])
        
    end
    
end

@btime mt_mzi($ro_x_co_x_an)

@btime mt_mea($ro_x_co_x_an)

;

In [ ]:
# Using a function to barrier the code that loops a `Matirx`.

function _mazi(da)
    
    ma = Matrix(da)
    
    mazi(ma)
    
end

function _maea(da)
    
    ma = Matrix(da)
    
    maea(ma)
    
end

@btime _mazi($ro_x_co_x_an)

@btime _maea($ro_x_co_x_an)

;

In [ ]:
# Barrier both.

function mt_mazi(da)
    
    ma = mt(da)
    
    mazi(ma)
    
end

function mt_maea(da)
    
    ma = mt(da)
    
    maea(ma)
    
end

@btime mt_mazi($ro_x_co_x_an)

@btime mt_maea($ro_x_co_x_an)

;

The compiler optimizes code at function boundaries.
Use multiple smaller functions! 

In [ ]:
@code_warntype mt_mea(ro_x_co_x_an)

In [ ]:
@code_warntype mt_maea(ro_x_co_x_an)

In [ ]:
@code_warntype maea(ma)